# 10.3 실습: 의미 검색 구현하기
### 학습 목표
- sentence-transformers 와 faiss 라이브라리를 활용해 의미 검색 semantic search 을 직접 구현해봅니다.
- 밀집 임베딩을 이용해 문장이나 문서의 이미를 고려한 검색을 수행합니다. (단순한 키워드 검색 아님)

### faiss 라이브러리
- 백터 연산 라이브러리 (코사인 유사도 계산 등등을 지원)
- 임베딩 백터 사이의 거리를 계산하여 가까운 임베딩을 찾습니다.

## 10.3.1 의미 검색 구현하기
9장에서 장마 기간 조회했던 KLUE 데이터셋을 사용해 의미 검색합니다.

https://huggingface.co/datasets/klue/klue/viewer/mrc?row=0

### 의미 검색
의미 검색의 장점: 키워드가 동일하지 않아도 유사하면 찾을 수 있습니다.

In [ ]:
import faiss
from datasets import load_dataset
from sentence_transformers import SentenceTransformer

# 데이터셋과 임베딩 모델 불러오기
klue_mrc_dataset = load_dataset('klue', 'mrc', split='train')
sentence_model = SentenceTransformer('snunlp/KR-SBERT-V40K-klueNLI-augSTS')

# 데이터셋 1000개 데이터만 추출해 텍스트 임베딩으로 변환
klue_mrc_dataset = klue_mrc_dataset.train_test_split(train_size=1000, shuffle=False)['train']
embeddings = sentence_model.encode(list(klue_mrc_dataset['context']))
print(embeddings.shape)
## (1000, 768)

# 인덱스 만들기
index = faiss.IndexFlatL2(embeddings.shape[1])
## 인덱스에 임베딩 저장하기
index.add(embeddings)

# 검색 쿼리 문장을 문자 임베딩으로 변환하고 인덱스에서 검색
query = "이번 연도에는 언제 비가 많이 올까?"
query_embedding = sentence_model.encode([query])
distances, indices = index.search(query_embedding, 3)

for idx in indices[0]:
    print(klue_mrc_dataset['context'][int(idx)][:50])
## 올여름 장마가 17일 제주도에서 시작됐다. 서울 등 중부지방은 예년보다 사나흘 정도 늦은
## 연구 결과에 따르면, 오리너구리의 눈은 대부분의 포유류보다는 어류인 칠성장어나 먹장어, 그
## 연구 결과에 따르면, 오리너구리의 눈은 대부분의 포유류보다는 어류인 칠성장어나 먹장어, 그

# 약 30초 소요

### faiss 사용 실패로 miniconda를 사용해 환경 구축하기
- python 최신 버전 3.13에서 pip로 faiss 설치 안됨 --> 최신 python에서는 지원이 안되서...
- pyenv 로 3.11 설치 후 실행 안됨 --> Faiss의 내부 스레드 관리와 Python 멀티프로세싱 간 충돌이 나서 실행 못함
- miniconda로 환경 구축해서 해결

```shell
# faiss import 했을 때, 실행이 정상적으로 안됨
$ rm -rf ~/.cache/huggingface/datasets/klue
$ pip install transformers==4.40.1 datasets==2.19.0 sentence-transformers==2.7.0 faiss-cpu==1.8.0 llama-index==0.10.34 llama-index-embeddings-huggingface==0.2.0
$ python semantic_search.py
[1]    48452 segmentation fault  python semantic_search.py
/Users/nhn/.pyenv/versions/3.11.9/lib/python3.11/multiprocessing/resource_tracker.py:254: UserWarning: resource_tracker: There appear to be 1 leaked semaphore objects to clean up at shutdown
  warnings.warn('resource_tracker: There appear to be %d '

# Python 패키지 관리자인 conda를 사용하여 환경 구축
$ brew install miniconda
$ conda init zsh
$ source ~/.zshrc
$ conda install -c conda-forge faiss-cpu
$ conda create -n ai-study python=3.11
$ conda activate ai-study
$ conda install -c conda-forge faiss-cpu
$ python --version
# 나중에 같은 환경이 필요하면
# $ echo 'alias use-ai-study="conda activate ai-study && export PATH=\"/opt/homebrew/Caskroom/miniconda/base/envs/ai-study/bin:\$PATH\""' >> ~/.zshrc
# $ use-ai-study
```

### 의미 검색의 한계
의미 검색의 단점: 관련성이 떨어지는 검색 결과가 나오기도 합니다.
질문인 로버트 헨리 딕과 태평양 전쟁에 대한 데이터는 관련된 내용이 전혀 없습니다.

In [ ]:
import faiss
from datasets import load_dataset
from sentence_transformers import SentenceTransformer

# 데이터셋과 임베딩 모델 불러오기
klue_mrc_dataset = load_dataset('klue', 'mrc', split='train')
sentence_model = SentenceTransformer('snunlp/KR-SBERT-V40K-klueNLI-augSTS')

# 데이터셋 1000개 데이터만 추출해 텍스트 임베딩으로 변환
klue_mrc_dataset = klue_mrc_dataset.train_test_split(train_size=1000, shuffle=False)['train']
embeddings = sentence_model.encode(list(klue_mrc_dataset['context']))
print(embeddings.shape)
## (1000, 768)

# 인덱스 만들기
index = faiss.IndexFlatL2(embeddings.shape[1])
## 인덱스에 임베딩 저장하기
index.add(embeddings)

# 검색 쿼리 문장을 문자 임베딩으로 변환하고 인덱스에서 검색
query = klue_mrc_dataset[3]['question']
## 로버트 헨리 딕이 1946년에 매사추세츠 연구소에서 개발한 것은 무엇인가?
print(query)
query_embedding = sentence_model.encode([query])
distances, indices = index.search(query_embedding, 3)

for idx in indices[0]:
    print(klue_mrc_dataset['context'][int(idx)][:50])
## 태평양 전쟁 중 뉴기니 방면에서 진공 작전을 실시해 온 더글러스 맥아더 장군을 사령관으로
## 태평양 전쟁 중 뉴기니 방면에서 진공 작전을 실시해 온 더글러스 맥아더 장군을 사령관으로
## 미국 세인트루이스에서 태어났고, 프린스턴 대학교에서 학사 학위를 마치고 1939년에 로체스

# 약 20초 소요

## 10.3.2 라마인덱스에서 Sentence-Transformers 모델 사용하기
라마인덱스에서는 기본적으로 OpenAI의 text-embedding-ada-002를 사용합니다.
라마인덱스에서 Sentence-Transformers 라이브러를 활용해 임베딩을 수행해보기

In [ ]:
from datasets import load_dataset
from llama_index.core import VectorStoreIndex, ServiceContext
from llama_index.core import Document, Settings
from llama_index.embeddings.huggingface import HuggingFaceEmbedding

klue_mrc_dataset = load_dataset('klue', 'mrc', split='train')

### ServiceContext Deprecated로 아래와 같이 수정하였습니다.
Settings.llm = None  # LLM을 사용하지 않으므로 None으로 설정
Settings.embed_model = HuggingFaceEmbedding(model_name="snunlp/KR-SBERT-V40K-klueNLI-augSTS")

text_list = klue_mrc_dataset[:100]['context']
documents = [Document(text=t) for t in text_list]

index_llama = VectorStoreIndex.from_documents(documents)
###

###
# embed_model = HuggingFaceEmbedding(model_name="snunlp/KR-SBERT-V40K-klueNLI-augSTS")
# service_context = ServiceContext.from_defaults(embed_model=embed_model, llm=None)
# ## LLM is explicitly disabled. Using MockLLM.
# # 로컬 모델 활용하기
# # service_context = ServiceContext.from_defaults(embed_model="local")
#
# text_list = klue_mrc_dataset[:100]['context']
# documents = [Document(text=t) for t in text_list]
#
# index_llama = VectorStoreIndex.from_documents(
#     documents,
#     service_context=service_context,
# )
###

print(klue_mrc_dataset[0]['question'])
## 북태평양 기단과 오호츠크해 기단이 만나 국내에 머무르는 기간은?

retrieval_engine = index_llama.as_retriever(similarity_top_k=5, verbose=True)
response = retrieval_engine.retrieve(
    klue_mrc_dataset[0]['question'],
)
print(len(response))
## 3
print(response[0].node.text)
## 태평양 전쟁 중 뉴기니 방면에서 진공 작전을 실시해 온 더글러스 맥아더 장군을 사령관으로 하는 미 육군 주체의 연합군 남서 태평양 방면군은 1944년 후반 마침내 필리핀을 진공하기로 결정했다. 그 첫 단계로 필리핀 방면의 전략 거점의 확보가 필요하였으며, 뉴기니 서쪽에 위치한 말루쿠 제도의 모로타이 섬을 공격 목표로 정했다. 또한 동시에 팔라우 제도의 펠렐리우 섬과 앙가우르 섬에도 미국 해군 주도의 연합국 중부 태평양 방면군이 공략을 맡았다.(이때의 전략 결정의 경위에 대해서는 필리핀 전투 (1944 - 1945)#미국을 참조.)
## 한편, 1942년에 네덜란드령 동인도의 일부였던 모로타이 섬을 점령한 일본군은 이후 수비 부대를 증강배치하지 않았다. 1944년 말루쿠 제도 방면의 방비 강화를 도모하고자 파견된 제32사단은 평야가 많은 비행장 건설에 적합한 주변의 할마헤라 섬을 방어의 중심으로 여겼다. 따라서 모로타이 섬에는 제32사단의 2개 대대가 비행장 건설을 추진했지만, 배수가 좋지 않아 건설을 포기했다. 이 2개 대대가 할마헤라 섬으로 철수한 이후에는 카와시마 타케노부(川島威伸) 중위를 지휘관으로 하는 제2유격대 소속의 2개 중대(주로 다카사고의용대)만 배치되어 있었다.
## 연합군이 상륙했을 때, 섬에는 9000명의 현지인이 살고 있었다. 도민에 대한 선무공작을 수행하기 위해 연합군의 상륙 부대에는 네덜란드 군 민정반이 추가 되었다.

# 약 15초 소요

원하는 임베딩을 선택할 수 있습니다.
예제에서는 LLM을 None으로 설정해 이상한 대답이 나옵니다.

# 10.4 검색 방식을 조합해 성능 높이기
의미 검색의 한계를 보완해서 키워드 검색과 조합하는 하이브리드 검색을 구현해볼겁니다.
- 키워드 검색: 동일한 키워드만 검색되기 때문에 관련성이 높은 것이 나오나, 유사한 내용은 조회를 못함
- 의미 검색: 유사한 내용은 검색이 되나, 관련성이 떨어지는 검색 결과가 나타날 수 있음

그 전에 키워드 검색 방식을 알아봅니다.

## 10.4.1 키워드 검색 방식: BM25
TF-IDF에서 진화된 형태로 계산량이 적으면서 뛰어난 성능을 보여 Elasticsearch의 기본 알고리즘으로 사용됩니다.

### TF-IDF
데이터를 숫자로 변환 할 때 '어느 문서에나 나오는 단어' 문제를 보완한 방식
TF-IDF(w) = TF(w) x log(N/DF(w))
- TF(w): 특정 문서에서 특정 단어 w가 등장한 횟수 (Term Frequency)
- DF(w) 특정 단어 w가 등장한 문서 수 (Inverse Document Frequency)

'은/는/이/가' 조사는 모든 문서에 등장하므로 log(N/DF(w))가 0이 되어 점수가 0이 됩니다.

### BM25 수식
$$\sum _{i=1}^{n}{\text{IDF}}(q_{i})\cdot {\frac {f(q_{i},D)\cdot (k_{1}+1)}{f(q_{i},D)+k_{1}\cdot \left(1-b+b\cdot {\frac {|D|}{\text{avgdl}}}\right)}}$$

### BM25의 IDF 계산식
$${\text{IDF}}(q_{i})=\ln \left({\frac {N-n(q_{i})+0.5}{n(q_{i})+0.5}}+1\right)$$

<table>
    <thead>
        <tr>
            <td style="text-align: center;"></td>
            <td style="text-align: center;">TF-IDF</td>
            <td style="text-align: center;">BM25</td>
        </tr>
    </thead>
    <tbody>
        <tr>
            <td  style="text-align: left;">공통점</td>
            <td  style="text-align: left;" colspan="2">
                * 특정 문서 내에 특정 토큰(q)이 많이 등장할수록 해당 토큰의 중요도가 높아집니다.<br/>
                * 전체 문서에서 특정 토큰(q)이 많이 등장할수록 해당 토큰의 중요도는 낮아집니다.
            </td>
        </tr>
        <tr>
            <td style="text-align: left;">차이점(1):<br/>포화효과</td>
            <td style="text-align: left;">* TF-IDF에서는 특정 문서내에서 토큰이<br/>자주 나오는 만클 중요도가 높아집니다.</td>
            <td style="text-align: left;">* BM25는 특정 문서 내에 토킨이 자주 나오더라도<br/> TF 항이 일정 값 이상으로 커지지 않습니다.</td>
        </tr>
        <tr>
            <td style="text-align: left;">차이점(2):<br/>문서 길이 고려</td>
            <td style="text-align: left;">* TF-IDF에서는 문서의 길이와 관계없이 점수를<br/> 산정합니다.</td>
            <td style="text-align: left;">* BM25는 짧은 문서에서 특정 토큰이 등장하는 경우<br/> 중요도를 높게 판단합니다.</td>
        </tr>
    </tbody>
</table>

### 개발자가 선택할 수 있는 설정값 K1과 b
- k1: 단어 빈도에 대한 포화 효과를 주는 값
  토큰이 자주 나왔을 때 최대 얼만큼 중요도를 줄지에 대한 수치
- b: 문서 길이 효과 반영 비율

## 10.4.2 상호 순위 조합 이해하기
- 하이브리드 검색을 위해 통계 기반 점수(키워드)와 임베딩 유사도 점수를 하나로 합쳐야 합니다.
- 단순하게 점수를 더하지 않고 상호 순위 조합 Reciprocal Rank Fusion (이하 RRF)를 활용합니다.

계산식: 점수 = 1(k+순위) + 1(k+순위)
k는 조절가능인자
--> 두 개 높은 순위인 것을 고릅니다.

![image.png](resources/10_4_2.png)

# 10.5 실습: 하이브리드 검색 구현하기
<span style="color: red;">하이브리드 검색</span> 구현합니다.

## BM25 구현하기

In [ ]:
import math
import numpy as np
from typing import List
from transformers import PreTrainedTokenizer
from collections import defaultdict

class BM25:
    def __init__(self, corpus:List[List[str]], tokenizer:PreTrainedTokenizer):
        self.tokenizer = tokenizer
        self.corpus = corpus
        # 토큰화된 텍스트 데이터
        self.tokenized_corpus = self.tokenizer(corpus, add_special_tokens=False)['input_ids']
        # 문서 수
        self.n_docs = len(self.tokenized_corpus)
        self.avg_doc_lens = sum(len(lst) for lst in self.tokenized_corpus) / len(self.tokenized_corpus)
        self.idf = self._calculate_idf()
        self.term_freqs = self._calculate_term_freqs()

    # 단어가 등장한 문서 수를 idf에 집계
    def _calculate_idf(self):
        idf = defaultdict(float)
        for doc in self.tokenized_corpus:
            for token_id in set(doc):
                idf[token_id] += 1
        for token_id, doc_frequency in idf.items():
            idf[token_id] = math.log(((self.n_docs - doc_frequency + 0.5) / (doc_frequency + 0.5)) + 1)
        return idf

    # n_docs(문서 수) 만큼 딕셔너리를 만들고, 각 문서 내 어떤 토큰이 몇 번 등장하는지 집계
    def _calculate_term_freqs(self):
        term_freqs = [defaultdict(int) for _ in range(self.n_docs)]
        for i, doc in enumerate(self.tokenized_corpus):
            for token_id in doc:
                term_freqs[i][token_id] += 1
        return term_freqs

    # 앞서 만든 idf와 term_freqs를 통해 검색하려는 쿼리와 각 문서 사이의 점수를 계산
    def get_scores(self, query:str, k1:float = 1.2, b:float=0.75):
        query = self.tokenizer([query], add_special_tokens=False)['input_ids'][0]
        scores = np.zeros(self.n_docs)
        for q in query:
            idf = self.idf[q]
            for i, term_freq in enumerate(self.term_freqs):
                q_frequency = term_freq[q]
                doc_len = len(self.tokenized_corpus[i])
                score_q = idf * (q_frequency * (k1 + 1)) / ((q_frequency) + k1 * (1 - b + b * (doc_len / self.avg_doc_lens)))
                scores[i] += score_q
        return scores

    # 점수가 가장 높은 k개의 문서의 인덱스와 점수를 반환
    def get_top_k(self, query:str, k:int):
        scores = self.get_scores(query)
        top_k_indices = np.argsort(scores)[-k:][::-1]
        top_k_scores = scores[top_k_indices]
        return top_k_scores, top_k_indices

### 단어 검색 테스트

In [ ]:
from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained('klue/roberta-base')

bm25 = BM25(['안녕하세요', '반갑습니다', '안녕 서울'], tokenizer)
print(bm25.get_scores('안녕'))
## [0.44713859 0.         0.52354835]

### 제대로 검색이 안되는 경우
정확한 키워드가 없이 이상한 결과가 검색됩니다.

In [ ]:
from transformers import AutoTokenizer
from datasets import load_dataset
tokenizer = AutoTokenizer.from_pretrained('klue/roberta-base')

klue_mrc_dataset = load_dataset('klue', 'mrc', split='train')

bm25 = BM25(list(klue_mrc_dataset['context']), tokenizer)

query = "이번 연도에는 언제 비가 많이 올까?"
_, bm25_search_ranking = bm25.get_top_k(query, 100)

for idx in bm25_search_ranking[:3]:
    print(klue_mrc_dataset['context'][int(idx)][:50])
## 케이팝 팬덤을 위한 어플리케이션 ‘블립’ 조사 결과, NCT 팬들이 가장 많이 입덕한 노래
## 케이팝 팬덤을 위한 어플리케이션 ‘블립’ 조사 결과, NCT 팬들이 가장 많이 입덕한 노래
## ‘현대무용 같지 않다.’ 오는 15일까지 서울 서초동 예술의전당 자유소극장 무대에 오르는

# 약 15초 소요

### 일치하는 키워드로 관련된 기사를 잘 찾은 케이스

In [ ]:
from transformers import AutoTokenizer
from datasets import load_dataset
tokenizer = AutoTokenizer.from_pretrained('klue/roberta-base')

klue_mrc_dataset = load_dataset('klue', 'mrc', split='train')

bm25 = BM25(list(klue_mrc_dataset['context']), tokenizer)

query = klue_mrc_dataset[3]['question']
print(query)
## 로버트 헨리 딕이 1946년에 매사추세츠 연구소에서 개발한 것은 무엇인가?
_, bm25_search_ranking = bm25.get_top_k(query, 100)

for idx in bm25_search_ranking[:3]:
    print(klue_mrc_dataset['context'][int(idx)][:50])
## 미국 세인트루이스에서 태어났고, 프린스턴 대학교에서 학사 학위를 마치고 1939년에 로체스
## 잭슨은 영국의 컴벌랜드 카운티에서 태어나 부모가 죽은 후에, 사우스캐롤라이나의 찰스턴에 이
## 영국과 북미 식민지 간의 관계가 악화되어 1775년 4월에 뉴잉글랜드에서 렉싱턴 콩코드 전

# 약 15초 소요

## 10.5.2 상호 순위 조합 구현하기

In [ ]:
from collections import defaultdict
from typing import List

def reciprocal_rank_fusion(rankings:List[List[int]], k=5):
    rrf = defaultdict(float)
    for ranking in rankings:
        for i, doc_id in enumerate(ranking, 1):
            rrf[doc_id] += 1.0 / (k + i)
    return sorted(rrf.items(), key=lambda x: x[1], reverse=True)

rankings = [[1, 4, 3, 5, 6], [2, 1, 3, 6, 4]]
print(reciprocal_rank_fusion(rankings))
## [(1, 0.30952380952380953), (3, 0.25), (4, 0.24285714285714285), (6, 0.2111111111111111), (2, 0.16666666666666666), (5, 0.1111111111111111)]

### 하이브리드 검색 구현하기

In [ ]:
import faiss
from datasets import load_dataset
from sentence_transformers import SentenceTransformer
from transformers import AutoTokenizer

# 검색 쿼리 문장 임베딩 변환, 인덱스 검색
def dense_vector_search(query:str, k:int):
    sentence_model = SentenceTransformer('snunlp/KR-SBERT-V40K-klueNLI-augSTS')
    query_embedding = sentence_model.encode([query])
    embeddings = sentence_model.encode(list(klue_mrc_dataset['context']))
    index = faiss.IndexFlatL2(embeddings.shape[1])
    index.add(embeddings)
    distances, indices = index.search(query_embedding, k)
    return distances[0], indices[0]

# 검색 쿼리 문장과 상호 순위 조합에 사용할 파라미터 받아 의미검색과 bm25 키워드 검색, 순위를 조합해 반환
def hybrid_search(query, k=20):
    _, dense_search_ranking = dense_vector_search(query, 100)
    tokenizer = AutoTokenizer.from_pretrained('klue/roberta-base')
    bm25 = BM25(list(klue_mrc_dataset['context']), tokenizer)
    _, bm25_search_ranking = bm25.get_top_k(query, 100)

    results = reciprocal_rank_fusion([dense_search_ranking, bm25_search_ranking], k=k)
    return results

klue_mrc_dataset = load_dataset('klue', 'mrc', split='train')

query = "이번 연도에는 언제 비가 많이 올까?"
print("검색 쿼리 문장: ", query)
## 검색 쿼리 문장:  이번 연도에는 언제 비가 많이 올까?
results = hybrid_search(query)
for idx, score in results[:3]:
    print(klue_mrc_dataset['context'][int(idx)][:50])
## 다음달엔 평년에 비해 때 이른 무더위가 기승을 부릴 전망이다. 8월에는 대기불안정과 저기압
## 올여름 장마가 17일 제주도에서 시작됐다. 서울 등 중부지방은 예년보다 사나흘 정도 늦은
## 올 들어 한반도 날씨가 수상쩍다. 23일 하루 동안 서울 등 중부지방엔 호우특보와 폭염특보

print("=" * 80)
query = klue_mrc_dataset[3]['question']
print("검색 쿼리 문장: ", query)
## 검색 쿼리 문장:  로버트 헨리 딕이 1946년에 매사추세츠 연구소에서 개발한 것은 무엇인가?

results = hybrid_search(query)
for idx, score in results[:3]:
    print(klue_mrc_dataset['context'][int(idx)][:50])
## 미국 세인트루이스에서 태어났고, 프린스턴 대학교에서 학사 학위를 마치고 1939년에 로체스
## 로스앤젤레스에서 성공적 비지니스맨 해리 프랜시스 홀더먼과 주부 캐서린 엘리자베스 로빈스의
## 로스앤젤레스에서 성공적 비지니스맨 해리 프랜시스 홀더먼과 주부 캐서린 엘리자베스 로빈스의

# 약 5분 소요

# 10.6 정리
- 텍스트의 의미를 숫자로 변환하는 다양한 기술 살펴봤습니다.
- 문장 임베딩 방식
- 의미 검색과 키워드 검색, 하이브리드 검색

# 출처
- https://en.wikipedia.org/wiki/Okapi_BM25